<a href="https://colab.research.google.com/github/hank199599/data_science_from_scratch_reading_log/blob/main/Chapter19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 深度學習(Deep learning)
原本指的是「深度」神經網路，現被用來泛指各種神經網路。

# 張量
在神經網路函示庫中，n維陣列被稱為**張量(tensor)**  
理想情況下，可以使用：

```python
# 張量Tensor要不是一個浮點數，就是一個張量列表
Tensor = Union[float,List[Tensor]]
```


In [1]:
#如同我們說：
Tensor = list

### 輔助函式：找出張量的形狀

In [2]:
from typing import List

def shape(tensor:Tensor) ->List[int]:
  sizes:List[int] = []
  while isinstance(tensor,list):
    sizes.append(len(tensor))
    tensor = tensor[0]
  return sizes

In [3]:
assert shape([1,2,3]) == [3]

In [4]:
assert shape([[1,2],[3,4],[5,6]]) == [3,2]

由於張量可能具有任意數量的維度，  
因此在使用張量時，通常需要採用**遞迴**的作法。

In [5]:
def is_1d(tensor:Tensor) ->bool:
  """
  如果tensor[0]是一個列表，他就是一個高維張量
  否則就是一個一維向量
  """
  return not isinstance(tensor[0],list)

In [6]:
assert is_1d([1,2,3])

In [7]:
assert not is_1d([[1,2],[3,4]])

### 輔助函式：tensor_sum 函數

In [8]:
def tensor_sum(tensor:Tensor)->float:
  """把張量的所有值加總起來"""
  if is_1d(tensor):
    return sum(tensor) #只是一個浮點數列表，就用Python的sum函式
  else:
    return sum(tensor_sum(tensor_i) for tensor_i in tensor)

In [9]:
assert tensor_sum([1,2,3]) == 6

In [10]:
assert tensor_sum([[1,2],[3,4]]) == 10

### 輔助函式：把某個函數套用到單一張量中的每個元素

In [11]:
from typing import Callable

def tensor_apply(f:Callable[[float],float],tensor:Tensor) ->Tensor:
  """把函式f套用到每個元素"""
  if is_1d(tensor):
    return [f(x) for x in tensor]
  else:
    return [tensor_apply(f,tensor_i) for tensor_i in tensor]

In [12]:
assert tensor_apply(lambda x:x+1,[1,2,3]) == [2,3,4]

In [13]:
assert tensor_apply(lambda x:2*x,[[1,2],[3,4]]) == [[2,4],[6,8]]

### 輔助函式：依據張量形狀，建立另一個形狀一樣的零張量

In [14]:
def zero_like(tensor:Tensor) ->Tensor:
  return tensor_apply(lambda _: 0.0,tensor)

In [15]:
assert zero_like([1,2,3]) == [0,0,0]

In [16]:
assert zero_like([[1,2],[3,4]]) == [[0,0],[0,0]]

### 輔助函式：將某個函式套用到兩個張量

In [17]:
def tensor_combine(f:Callable[[float,float],float],t1:Tensor,t2:Tensor) ->Tensor:
  """把函式f套用到t1與t2的相應元素"""
  if is_1d(t1):
    return [f(x,y) for x,y in zip(t1,t2)]
  else:
    return [tensor_combine(f,t1_i,t2_i) for t1_i,t2_i in zip(t1,t2)]

In [18]:
import operator
assert tensor_combine(operator.add,[1,2,3],[4,5,6]) == [5,7,9]
assert tensor_combine(operator.mul,[1,2,3],[4,5,6]) == [4,10,18]

# 層的抽象概念

建立一種機制，能用來時做出各式各樣的神經網路。
最基本的概念是「Layer」。  
他知道如何把輸入套入某種函數，亦能夠進行**反向傳播**的方式計算梯度。
  
在真正的子類別中，forward 與 backward 方法都會進行實作，  
一旦建構了神經網路，我們就能以梯度遞減的方式來進行訓練。



In [19]:
from typing import Iterable,Tuple

class Layer:
  """
  我們的神經網路是由許多層組成，其中每一層都知道
  如何以正向傳播的方式對輸入進行某些計算
  以及如何以反向傳播的方式計算梯度
  """
  def forward(self,input):
    """
    可以注意到，這裡缺了型別可以設定。
    我們並不打算限定各層的輸入是什麼型別。
    也不限定return的輸入是甚麼型別
    """
    raise NotImplementedError
  
  def backward(self,gradient):
    """
    同樣地，我們並不打算限定梯度應該是什麼型別，
    這完全由你自己決定，
    只要確定合理即可
    """
    raise NotImplementedError
  
  def params(self)->Iterable[Tensor]:
    """
    返回此層的參數，預設的實作方式不會回送任何東西。
    如果你的Layer沒有任何參數，
    並不需要實做這個方法
    """
    return ()
  def grads(self) ->Iterable[Tensor]:
    """
    送回梯度，順序與params相同
    """
    return ()

### 範例：不須更新參數的sigmoid層：
在正向傳遞中，保存了sigmoid函式的計算結果。  
以供後續的反向傳遞使用。


In [20]:
from typing import List
import math

Vector = List[float]

def sigmoid(t:float)->float:
  return 1/(1+math.exp(-t))

class Sigmoid(Layer):

  def forward(self,input:Tensor)->Tensor:
    """
    把sigmoid函式套用到輸入張量的每個元素，
    然後儲存結果，供反向傳播使用
    """
    self.sigmoids = tensor_apply(sigmoid,input)
    return self.sigmoids
  
  def backward(self,gradient:Tensor)->Tensor:
    return tensor_combine(
        lambda sig,grad:sig*(1-sig)*grad,
        self.sigmoids,
        gradient)


## 線性層
代表神經元的 **dot(weight,inputs)** 點積的部分
  
在這裡實作三種方式生成初始的隨機張量：
1. 自[0,1]的隨機分布中選擇初始值
2. 自標準常態分布中選擇初始值
3. 使用「Xavier initialization」，其中每個權重為平均是0，變異量為2/(num_inputs+num_outputs)的標準常態分布中選擇初始值


### 前置作業

In [21]:
import random
import math

SQRT_TWO_PI = math.sqrt(2* math.pi)

def normal_pdf(x:float,mu:float=0,sigma:float=1) ->float:
  return (math.exp(-(x-mu)**2/2/sigma**2)/(SQRT_TWO_PI*sigma))

def normal_cdf(x:float,mu:float=0,sigma:float=1)->float:
  return (1+math.erf((x-mu)/math.sqrt(2)/sigma))/2

def inverse_normal_cdf(p:float,
            mu:float = 0,
            sigma:float=1,
            tolerance:float=0.00001) -> float:
  # 如果不是標準常態分佈，就先轉換成標準常態分佈
  if mu != 0 or sigma != 1:
    return mu + sigma*inverse_normal_cdf(p,tolerance = tolerance)
  
  low_z = -10.0 # normal_cdf(-10)是(趨近於) 0
  hi_z =  10.0 # normal_cdf(10)是(趨近於) 1
  while hi_z - low_z > tolerance:  
    mid_z = (low_z + hi_z) / 2   # 計算出中間值
    mid_p = normal_cdf(mid_z)     # 以及累積分佈函數在該處所應對的值
    if mid_p < p :
      low_z = mid_z        #中間的值太低，就往上繼續搜尋
    else:
      hi_z = mid_z        #中間的值太高，就往下繼續搜尋
  
  return mid_z

In [22]:
def random_uniform(*dims:int) ->Tensor:
  if len(dims) == 1:
    return [random.random() for _ in range(dims[0])]
  else:
    return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims:int,mean:float=0.0,variance:float=1.0) ->Tensor:
  if len(dims) == 1:
    return [mean + variance * inverse_normal_cdf(random.random()) for _ in range(dims[0])]
  else:
    return [random_normal(*dims[1:],mean=mean,variance=variance) for _ in range(dims[0])]
  

In [23]:
assert shape(random_uniform(2,3,4)) == [2,3,4]

In [24]:
assert shape(random_normal(5,6,mean=10)) == [5,6]

將他們包裝在random_tensor函式中

In [25]:
def random_tensor(*dims:int,init:str = 'normal')->Tensor:
  if init == 'normal':
    return random_normal(*dims)
  elif init == 'unifrom':
    return random_uniform(*dims)
  elif init == 'xavier':
    variance = len(dims)/sum(dims)
    return random_normal(*dims,variance=variance)
  else:
    raise ValueError(f"unknown init: {init}")

### 定義線性層

In [26]:
def dot(v:Vector,w:Vector)->float:
  #計算v_1*w_1+... +v_n*w_n
  assert len(v)==len(w),"兩個向量必須有相同的維度"

  return sum(v_i*w_i for v_i,w_i in zip(v,w))

In [27]:
class Linear(Layer):
  def __init__(self,input_dim:int,output_dim:int,init:str='xavier')->None:
    """
    此層具有output_dims個神經元，每個神經元都有 imnput_dims個權重以及一個偏差量
    """
    self.input_dim = input_dim
    self.output_dim = output_dim

    # self.w[o] 是第 o 個神經元的權重
    self.w = random_tensor(output_dim,input_dim,init=init)

    # self.w[o] 是第 o 個神經元的偏差項
    self.b = random_tensor(output_dim,init=init)

  def forward (self,input:Tensor) ->Tensor:
      
    """
    每個神經元都有一個輸出，將它保存在向量中
    = 輸入和權重的點積 + 偏差值
    """
    #保存以供反向傳遞使用
    self.input = input

    #送回神經元的輸出向量
    return [dot(input,self.w[o])+self.b[o] for o in range(self.output_dim)]

  def backward(self,gradient:Tensor) ->Tensor:
    #　每個　b[o]　都會被加到　output[o]　之中
    #　亦即ｂ的梯度等於　output的梯度
    self.b_grad = gradient

    # 每個w[o][i]都會乘以input[i]再加到output[o]之中
    # 因此梯度是 input[i]*gradient[o]
    self.w_grad = [[self.input[i]*gradient[o] for i in range(self.input_dim)] for o in range(self.output_dim)]

    # 每個input[i]都會與每個w[o][i]相乘，再加到每個output[o]之中
    # 因此其梯度就是橫跨所有outputs
    # w[o][i]*gradient[o]加總之和
    return [sum(self.w[o][i]* gradient[o] for o in range(self.output_dim)) for i in range(self.input_dim)]
  
  def paramas(self) ->Iterable[Tensor]:
    return [self.w,self.b]
  
  def grads(self)->Iterable[Tensor]:
    return [self.w_grad,self.b_grad]

# 把神經網路視為一系列的層

In [28]:
from typing import List

class Sequential(Layer):
  """
  一行就包括一系列的其他層。
  每一層的輸出做為下一層的輸入，
  """
  def __init__(self,layers:List[Layer])->None:
    self.layers = layers
  
  def forward(self,input):
    """讓輸入照順序正向通過每一層"""
    for layer in self.layers:
      input = layer.forward(input)
    return input
  
  def backward(self,gradient):
    """讓梯度反向通過每一層以進行反向傳播"""
    for layer in reversed(self.layers):
      gradient = layer.backward(gradient)
    return gradient
  
  def params(self) ->Iterable[Tensor]:
    """送回每一層的參數"""
    return (param for layer in self.layers for param in layer.params())

  def grads(self) ->Iterable[Tensor]:
    """送回每一層的梯度"""
    return (grad for layer in self.layers for grad in layer.grads())

將XOR神經網路表示為：

In [29]:
xor_net=Sequential([
  Linear(input_dim=2,output_dim=2),
  Sigmoid(),
  Linear(input_dim=2,output_dim=1),
  Sigmoid()])

# 損失與最佳化

### 定義名為「Loss」的抽象類別

In [30]:
class Loss:
  def loss(self,predicted:Tensor,actual:Tensor) ->float:
    """計算我們預測結果的程度"""
    raise NotImplementedError
  
  def gradient(self,predicted:Tensor,actual:Tensor) ->Tensor:
    """如果預測改變，則損失會隨之改變"""
    raise NotImplementedError

### 利用**平方誤差和**做為損失函數

In [31]:
class SSE(Loss):
  """計算平方誤差和，以作為損失函數"""
  def loss(self,predicted:Tensor,actual:Tensor)->float:

    #計算出平方誤差量
    squared_errors = tensor_combine(
      lambda predicted,actual:(predicted - actual) ** 2,
      predicted,
      actual)

    #接著全部加總起來
    return tensor_sum(squared_errors)
  
  def gradient(self,predicted:Tensor,actual:Tensor)->Tensor:
    return tensor_combine(
      lambda predicted,actual: 2 * (predicted - actual),
      predicted,
      actual)

### 梯度遞減的實作
以往是以人工方式完成所有梯度遞減的操作，然而在這裡不太適用。
1. 神經網路有多個參數需要更新
2. 希望能使用更聰明的梯度遞減變形作法而無須重新改寫程式

In [32]:
class Optimizer:
  """
  Optimizer會更新layer的權重值，
  其根據不是來自layer就是來自Optimizer
  """
  def step(self,layer:Layer) ->None:
    raise NotImplementedError

利用Optimizer來實作梯度遞減



In [33]:
class GradientDescent(Optimizer):
  def __init__(self,learning_rate:float=0.1)->None:
    self.lr = learning_rate
  
  def step(self,layer:Layer) ->None:
    for param,grad in zip(layer.params(),layer.grads()):
      #運用梯度遞減的方式更新參數
      param[:] = tensor_combine(
        lambda param,grad:param - grad * self.lr,
        param,
        grad
      )

### 片段指定值(slice assignment)
**反映一個事實**：重新指定列表的值，並不會改變其值

In [34]:
tensor= [[1,2],[3,4]]

for row in tensor:
  row = [0,0]

assert tensor == [[1,2],[3,4]] #一般指定值的方式不會更動到列表內容

for row in tensor:
  row[:] = [0,0]

assert tensor == [[0,0],[0,0]] #  片段指定值的方式就會更動到列表內容

### 動量(momentum)
保留先前梯度移動平均值，每得到新的梯度值就更新這個平均梯度值並往這個平均方向邁開一步。

In [35]:
class Momentum(Optimizer):
  def __init__(self,learning_rate = float,momentum:float=0.9) ->None:
    self.lr = learning_rate
    self.mo = momentum
    self.updates:List[Tensor]=[] #移動平均

  def step(self,layer:Layer)->None:
    #如果還沒有之前的更新值，就全部使用零來當作起始值
    if not self.updates:
      self.updates = [zero_like(grad) for grad in layer.grads()]
    
    for update,param,grad in zip(self.updates,layer.params(),layer.grads()):
      #套用動量的計算方式
      update[:] = tensor_combine(lambda u,g:self.mo*u + (1-self.mo)*g, update , grad)

      #接著執行梯度遞減的步驟
      param[:] = tensor_combine(lambda p,u:p-self.lr*u,param,update)

# 範例：XOR再次嘗試

重新建立訓練資料

In [36]:
xs = [[0.,0],[0.,1],[1.,0],[1.,1]]
ys = [[0.],[1.],[1.],[0.]]

定義網路  
(*在這裡可以省去最後的sigmoid層*)

In [40]:
random.seed(0)

net = Sequential([
  Linear(input_dim=2,output_dim=2),
  Sigmoid(),
  Linear(input_dim=2,output_dim=1)
])

定義一個簡易的定義迴圈

In [44]:
import tqdm

optimizer = GradientDescent(learning_rate=0.1)
loss = SSE()

with tqdm.trange(3000) as t:
  for epoch in t:
    epoch_loss = 0.0

    for x,y in zip(xs,ys):
      predicted = net.forward(x)
      #print(predicted)
      epoch_loss += loss.loss(predicted,y)
      gradient = loss.gradient(predicted,y)
      net.backward(gradient)

      optimizer.step(net)
    
    t.set_description(f"xor loss {epoch_loss:.3f}")

xor loss 1.597: 100%|██████████| 3000/3000 [00:03<00:00, 751.39it/s]


檢查訓練後的網路

In [42]:
for param in net.params():
  print(param)

從得到的網路來看，結果大致如下：
* hidden1 = -2.6 * x1 + -2.7 * x2 + 0.2
* hidden2 = 2.1 * x1 + 2.1 * x2 -3.4
* optput= -3.1 * h1 + -2.6 * x2 + 1.8

# 其他激活函數

# **範例**：FizzBuzz 再次嘗試

# Softmax與交叉熵

# Dropout隨機拋棄

# 範例：MNIST

# 模型的儲存與載入